In [74]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import copy

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [ ]:
def decode_data(file_path):
    # Đọc dữ liệu vào một numpy array
    data = np.fromfile(file_path, dtype=np.uint8)

    # Giả định kích thước mỗi bản ghi (có thể thay đổi tùy theo cấu trúc tệp thực tế)
    record_size = 32  # Giả định
    num_records = len(data) // record_size

    # Số lượng cột dữ liệu (bao gồm ngày, thời gian và các giá trị int32 còn lại)
    num_columns = record_size // 4  # Mỗi giá trị int32 chiếm 4 byte

    # Sử dụng numpy để cắt và giải mã dữ liệu hiệu quả hơn
    # Tạo một numpy array để chứa các giá trị int32 và float32
    raw_data = data.reshape(num_records, record_size // 4, 4)

    # Giải mã ngày và thời gian (int32) ở cột 0 và 1, các cột còn lại là float32
    int_data = raw_data[:, :2].view(np.int32)  # Giải mã int32 (2 cột)
    float_data = raw_data[:, 2:].view(np.float32)  # Giải mã float32 (các cột còn lại)

    # Kết hợp dữ liệu
    records = np.hstack((int_data, float_data))

    # Đảm bảo rằng dữ liệu là 2D
    records = records.reshape(num_records, num_columns)

    # Đảo ngược lại dữ liệu trước khi chuyển thành DataFrame
    records = records[::-1]

    # Chuyển đổi thành DataFrame và loại bỏ dòng đầu tiên
    df = pd.DataFrame(records, columns=[f"Col_{i}" for i in range(num_columns)])
    return df  # Loại bỏ đi dòng dữ liệu đầu tiên không cần thiết

def get_file_name_list(folder_path):
    file_name_list = []
    files = os.listdir(folder_path)
    for file in files:
        file_name_list.append(file[:-4])
    return file_name_list

def filter_market_file_name_list(file_name_list):
    filtered_list = [item for item in file_name_list if not (item.endswith('_AC') or item.endswith('_CC'))]
    return filtered_list

eod_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\stock"
eod_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\index"
eod_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\futures"
itd_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\stock"
itd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\index"
itd_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\futures"
nn_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\foreign"
td_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\prop"
nntd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\market"
other_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\other"

In [76]:
#Tạo dict map thời gian và số lượng cổ phiếu
period_map = pd.read_excel("xlsx_data/period_stock_list.xlsx", sheet_name='period_map')
period_map_dict = period_map.set_index('index').apply(lambda row: row.tolist(), axis=1).to_dict()

#Xoá đi quý hiện tại để chỉ tính toán tới quý trước đó
def get_quarter(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
        previous_quarter = "q4"
        previous_year = year - 1
    elif 4 <= month <= 6:
        quarter = "q2"
        previous_quarter = "q1"
        previous_year = year
    elif 7 <= month <= 9:
        quarter = "q3"
        previous_quarter = "q2"
        previous_year = year
    else:
        quarter = "q4"
        previous_quarter = "q3"
        previous_year = year
    
    if name == 'current_quarter':
        return f'{quarter}_{year}'
    if name == 'previous_quarter':
        return f'{previous_quarter}_{previous_year}'
        
#Lấy ra khoảng thời gian tính toán cho quý này và quý trước
calculate_time_span = [period_map_dict['q2_2020'][0], period_map_dict[get_quarter('current_quarter')][1]]
current_quarter_span = [period_map_dict[get_quarter('current_quarter')][0], period_map_dict[get_quarter('current_quarter')][1]]
previous_quarter_span = [period_map_dict[get_quarter('previous_quarter')][0], period_map_dict[get_quarter('previous_quarter')][1]]

#Lấy ra list cổ phiếu của giai đoạn hiện tại
period_stock_list = pd.read_excel("xlsx_data/period_stock_list.xlsx", sheet_name='period_stock_list')
current_stock_list = period_stock_list[get_quarter('current_quarter')].dropna().tolist()
total_stock_list = period_stock_list['all_stock'].dropna().tolist()

In [ ]:
def clean_eod_index_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200000]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'value_traded']
    #Điều chỉnh lại giá trị các cột
    df_clean['value_traded'] = df_clean['value_traded']/1000000000

    
    return df_clean.reset_index(drop=True)

def clean_futures_index_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200000]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'oi']
    
    return df_clean.reset_index(drop=True)

eod_index_dict = {}
for index in get_file_name_list(eod_index_folder_path):
    temp_file_path = eod_index_folder_path + f'\\{index}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_eod_index_data(temp_df_raw)
    temp_df_clean.insert(0, 'stock', index)
    eod_index_dict[index] = temp_df_clean

#Thêm giá dầu và DJI
for index in ['^DJI','^CLZ']:
    temp_file_path = other_index_folder_path + f'\\{index}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_eod_index_data(temp_df_raw)
    temp_df_clean.insert(0, 'stock', index)
    eod_index_dict[index[1:]] = temp_df_clean

#Thêm BTC và XAU
for index in ['^BTC','^XAU']:
    temp_file_path = other_index_folder_path + f'\\{index}' + '\\USD.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_eod_index_data(temp_df_raw)
    temp_df_clean.insert(0, 'stock', index)
    eod_index_dict[index[1:]] = temp_df_clean

eod_futures_dict = {}
for index in get_file_name_list(eod_futures_folder_path):
    temp_file_path = eod_futures_folder_path + f'\\{index}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_futures_index_data(temp_df_raw)
    temp_df_clean.insert(0, 'stock', index)
    eod_futures_dict[index] = temp_df_clean

In [78]:
def clean_itd_data(df_raw):
    #Lọc ra đúng 1 ngày dữ liệu cuối cùng
    df_raw = df_raw[df_raw['Col_0'] == max(df_raw['Col_0'])]
    #Tạo cột date-time mới từ 2 cột date và time cũ
    df_raw['date'] = df_raw['Col_0'].astype(int).astype(str) + ' ' + df_raw['Col_1'].astype(int).astype(str)
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_0', 'Col_1', 'Col_7'])
    #Sắp xếp lại thứ tự các cột
    df_clean = df_clean[['date'] + [f"Col_{i}" for i in range(2, len(df_clean.columns)+1)]]
    # #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['date'] = pd.to_datetime(df_clean['date'], format='%Y%m%d %H%M%S')
    # #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    #Làm tròn khung thời gian tới 5 phút
    df_clean['date'] = df_clean['date'].dt.floor('5T')
    df_clean = df_clean.set_index("date").resample("5T", closed='right', label='right').agg({    
        "open": "first",  
        "high": "max",  
        "low": "min", 
        "close": "last",  
        "volume": "sum"   
    }).dropna().reset_index()

    return df_clean.sort_values(by="date", ascending=False).reset_index(drop=True)

itd_index_dict = {}
for index in get_file_name_list(itd_index_folder_path):
    temp_file_path = itd_index_folder_path + f'\\{index}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_itd_data(temp_df_raw)
    temp_df_clean.insert(0, 'stock', index)
    itd_index_dict[index] = temp_df_clean
for index in get_file_name_list(itd_futures_folder_path):
    temp_file_path = itd_futures_folder_path + f'\\{index}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_itd_data(temp_df_raw)
    temp_df_clean.insert(0, 'stock', index)
    itd_index_dict[index] = temp_df_clean


In [79]:
#Khởi tạo vnindex_series để xác định ngày hiện tại
vnindex_series = eod_index_dict['VNINDEX'].sort_values('date', ascending=False).reset_index(drop=True)['date']

#Tạo date_series cho thời gian tính toán
date_series = pd.DataFrame(vnindex_series).rename(columns={0:'date'})
date_series = date_series[(date_series['date'] >= calculate_time_span[0]) & (date_series['date'] <= calculate_time_span[1])]

#Xác định ngày hiện tại
today = vnindex_series.iloc[0]

#Xác định thời gian hiện tại
current_time = itd_index_dict['HNXINDEX']['date'].iloc[0]

#Khởi tạo time_series bao gồm tất cả khung thời gian của ngày hiện tại
time_series_list = []
time_series_list.extend(pd.date_range(start=f'{today} 09:00:00', end=f'{today} 11:25:00', freq='5T'))
time_series_list.extend(pd.date_range(start=f'{today} 13:00:00', end=f'{today} 14:55:00', freq='5T'))
time_series = pd.DataFrame(time_series_list).rename(columns={0:'date'})

#Điều chỉnh lại time_series bỏ đi các hàng thời gian chưa có dữ liệu
time_series = time_series.loc[time_series['date'].dt.time <= current_time.time()].sort_values('date', ascending=False).reset_index(drop=True)

#Khởi tạo khung thời gian bắt đầu từ 9h15 để vẽ các biểu đồ
itd_series = pd.DataFrame(time_series_list[3:]).rename(columns={0:'date'}).sort_values('date', ascending=False)

data_13 = itd_series.merge(itd_index_dict['VNINDEX'], on='date', how='left').sort_values('date')
data_14 = itd_series.merge(itd_index_dict['VN30'], on='date', how='left').sort_values('date')
data_15 = itd_series.merge(itd_index_dict['VN30F1M'], on='date', how='left').sort_values('date')

In [116]:
def clean_nntd_index_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200000]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1', 'Col_3', 'Col_4'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'sell_volume', 'buy_volume', 'sell_value', 'buy_value']
    #Điều chỉnh lại giá trị các cột
    df_clean['buy_value'] = df_clean['buy_value']/1000000000
    df_clean['sell_value'] = -df_clean['sell_value']/1000000000
    df_clean['net_volume'] = df_clean['buy_volume'] - df_clean['sell_volume']
    df_clean['net_value'] = df_clean['buy_value'] + df_clean['sell_value']
    
    return df_clean.reset_index(drop=True)

nntd_futures_dict = {}
for index in get_file_name_list(nntd_index_folder_path):
    temp_file_path = nntd_index_folder_path + f'\\{index}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_nntd_index_data(temp_df_raw)
    temp_df_clean.insert(0, 'stock', index)
    nntd_futures_dict[index] = temp_df_clean

data_11 = pd.DataFrame([
    nntd_futures_dict['VN30F1M_NN'].iloc[0],
    nntd_futures_dict['VN30F1M_TD'].iloc[0],
    nntd_futures_dict['VN30F2M_NN'].iloc[0],
    nntd_futures_dict['VN30F2M_TD'].iloc[0]
])
data_11['luy_ke'] = [nntd_futures_dict['VN30F1M_NN']['net_volume'].iloc[:20][::-1].cumsum()[::-1].iloc[0],
                        nntd_futures_dict['VN30F1M_TD']['net_volume'].iloc[:20][::-1].cumsum()[::-1].iloc[0],
                        nntd_futures_dict['VN30F2M_NN']['net_volume'].iloc[:20][::-1].cumsum()[::-1].iloc[0],
                        nntd_futures_dict['VN30F2M_TD']['net_volume'].iloc[:20][::-1].cumsum()[::-1].iloc[0]
                    ]

data_12 = nntd_futures_dict['VN30F1M_NN'][['date','net_volume']].iloc[:20].rename(columns={'net_volume':'NN'})
data_12['TD'] = nntd_futures_dict['VN30F1M_TD']['net_volume'].iloc[:20]
data_12.loc[data_12.index[-1], ['NN', 'TD']] = 0
data_12['NN'] = data_12['NN'][::-1].cumsum()[::-1]
data_12['TD'] = data_12['TD'][::-1].cumsum()[::-1]

In [117]:
for index, df in eod_index_dict.items():
    df['pct_change'] = df['close'][::-1].pct_change()[::-1]
    df['value_change'] = df['close'][::-1].diff()[::-1]
    df['pct_5'] = df['close'][::-1].rolling(window=5).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
    df['pct_20'] = df['close'][::-1].rolling(window=20).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
    df['pct_60'] = df['close'][::-1].rolling(window=60).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]

data_1 = {}
for index, df in eod_index_dict.items():
    data_1[index] = df[['close','volume','pct_change','value_change','pct_5','pct_20','pct_60']].iloc[0]

data_1 = pd.DataFrame.from_dict(data_1, orient='index').reset_index()

In [118]:
week_groupby = eod_futures_dict['VN30F1M'].set_index("date").resample("W", closed='right', label='right').agg({    
    "open": "first",  
    "high": "max",  
    "low": "min", 
    "close": "last",  
    "volume": "sum"   
}).dropna().sort_values('date', ascending=False).reset_index().iloc[0]

month_groupby = eod_futures_dict['VN30F1M'].set_index("date").resample("M", closed='right', label='right').agg({    
    "open": "first",  
    "high": "max",  
    "low": "min", 
    "close": "last",  
    "volume": "sum"   
}).dropna().sort_values('date', ascending=False).reset_index().iloc[0]

In [119]:
data_2 = {}
data_2['oi'] = eod_futures_dict['VN30F1M'].iloc[0]['oi']
data_2['volume'] = eod_futures_dict['VN30F1M'].iloc[0]['volume']
data_2['basis'] = eod_futures_dict['VN30F1M'].iloc[0]['close'] - eod_index_dict['VN30'].iloc[0]['close']
data_2['open_month'] = month_groupby['open']
data_2['open_week'] = week_groupby['open']
data_2['highest_week'] = week_groupby['high']
data_2['lowest_week'] = week_groupby['low']
data_2['balanced_week'] = (week_groupby['high'] + week_groupby['low'])/2

data_2 = pd.DataFrame.from_dict(data_2, orient='index', columns=['value']).reset_index()

In [120]:
from sqlalchemy import create_engine, text

# Thông tin kết nối cơ sở dữ liệu
username = 'twan'
password = 'chodom'
database = 't2m'
host = '14.225.192.30'
port = '3306'
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")


data_3 = pd.read_sql("SELECT * FROM market_price_breath_df", con=engine).drop(columns='index')
data_4 = pd.read_sql("SELECT * FROM group_eod_score_liquidity_df", con=engine).drop(columns='index')
data_5 = pd.DataFrame([pd.read_sql("SELECT * FROM market_sentiment_df", con=engine).dropna()[['last_ratio','last_sentiment']].iloc[0]])
data_6 = pd.read_sql("SELECT * FROM market_top_stock_df", con=engine).drop(columns='index')

data_7 = pd.read_sql("SELECT * FROM market_index_card_df", con=engine).drop(columns='index')
data_7 = data_7[data_7['stock'].isin(['VNINDEX','VN30','VN30F1M'])]
data_7['value_traded'] = [eod_index_dict['VN30'].iloc[0]['value_traded'], 
                          eod_index_dict['VNINDEX'].iloc[0]['value_traded'], 
                          eod_futures_dict['VN30F1M'].iloc[0]['close']*100000*eod_futures_dict['VN30F1M'].iloc[0]['volume']/1000000000]

data_8 = pd.read_sql("SELECT * FROM group_ms_chart_df", con=engine).drop(columns='index')
data_8 = data_8[data_8['name']=='Thị trường'].iloc[:60].drop(columns=['name'])
data_8.columns = ['Ngày', 'Tuần', 'Tháng', 'Quý', 'Bán niên', '1 Năm', '2 Năm']

data_9 = pd.read_sql("SELECT * FROM nn_td_20p_df", con=engine).drop(columns='index')
data_9 = data_9[data_9['id']=='HSX'].sort_values('date')

data_10 = pd.read_sql("SELECT * FROM nn_td_buy_sell_df", con=engine).drop(columns='index')
data_10 = data_10[data_10['id']=='HSX'].iloc[:,:5]

In [121]:
# Define the file path to save the Excel file
file_path = "cts-report-data/cts-report-data.xlsx"

with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
    data_1.to_excel(writer, sheet_name='data_1', index=False)
    data_2.to_excel(writer, sheet_name='data_2', index=False)
    data_3.to_excel(writer, sheet_name='data_3', index=False)
    data_4.to_excel(writer, sheet_name='data_4', index=False)
    data_5.to_excel(writer, sheet_name='data_5', index=False)
    data_6.to_excel(writer, sheet_name='data_6', index=False)
    data_7.to_excel(writer, sheet_name='data_7', index=False)
    data_8.to_excel(writer, sheet_name='data_8', index=False)
    data_9.to_excel(writer, sheet_name='data_9', index=False)
    data_10.to_excel(writer, sheet_name='data_10', index=False)
    data_11.to_excel(writer, sheet_name='data_11', index=False)
    data_12.to_excel(writer, sheet_name='data_12', index=False)
    data_13.to_excel(writer, sheet_name='data_13', index=False)
    data_14.to_excel(writer, sheet_name='data_14', index=False)
    data_15.to_excel(writer, sheet_name='data_15', index=False)